In [1]:
import requests as requests
import urllib.parse
import json
import pandas as pd
import numpy as np
import time
from datetime import datetime
from datetime import timedelta

In [2]:
# param_1945 - кол-во комнат
# param_1957 - вторичка/новостройка
# param_2009 - тип дома
# param_2113 - этаж
# param_2213 - этажей в доме
# param_2313 - Площадь
# param_2314 - Адрес
# param_12721 - Площадь кухни, м²
# param_12722 - Жилая площадь, м²

cols = ['id','price','time','metro','address','room_quantity', 'new_old', 'house_type', 'floor_number', 
        'floor_quantity', 'space', 'kitchen_space', 'living_space','coords']

keys = ['id','price','time','metro','address','param_1945', 'param_1957', 'param_2009', 'param_2113', 
        'param_2213', 'param_2313', 'param_12721', 'param_12722','coords']

parameters = {'user' : 'polykova.olga98@gmail.com', 'token' : 'ec81436bf58a4b578bace05a0c28b381',
              'category_id' : 2,
              'nedvigimost_type' : 1,
              'city' : 'Москва',
              'date1' : '2018-04-01'}

In [13]:
earl_time = '2018-05-15'
flag = 0
print('Starting downloading dataset...')
while True:
    parameters['date2'] = earl_time
    response = requests.get('http://ads-api.ru/main/api?'+urllib.parse.urlencode(parameters))
    result = json.loads(response.content)['data']
    data = list()
    for r in result:
        temp = list()
        for i in range(len(cols)):
            if keys[i] in r.keys() : temp.append(r[keys[i]])
            else : temp.append(np.nan)
        data.append(temp) 
    if flag == 0:
        flag = 1
        dataset = pd.DataFrame(columns = cols, data = data)
    else:
        dataset2 = pd.DataFrame(columns = cols, data = data)
        dataset = pd.concat([dataset,dataset2], ignore_index = True)
    prev_time = earl_time
    earl_time = dataset.loc[len(dataset)-1,'time']
    if earl_time == prev_time : 
        earl_time = (datetime.strptime(earl_time,'%Y-%m-%d %H:%M:%S')-timedelta(0,1)).strftime('%Y-%m-%d %H:%M:%S')
    if len(result) < 50: break
    print('.',end='')
    time.sleep(5)
print('Finished!')
print('Dataset length: '+str(len(dataset)))

Starting downloading dataset...
........................................................................................................................................................................................................................................................Finished!
Dataset length: 152318


In [17]:
dataset['lang'] = dataset.coords.apply(lambda x: x['lng'])
dataset['lat'] = dataset.coords.apply(lambda x: x['lat'])
dataset = dataset.drop(['coords'], axis = 1)

In [21]:
dataset=dataset.drop_duplicates(keep='first')
dataset=dataset.drop_duplicates(keep='first', subset = ['price','metro','address','room_quantity', 'new_old','house_type', 
                                                        'floor_number', 'floor_quantity', 'space', 'kitchen_space', 
                                                        'living_space'])
print('Длина датасета после удаления дубликатов: ', len(dataset))

Длина датасета после удаления дубликатов:  136383


In [22]:
# Сохранение датасета в файл
dataset.to_csv('msk_dataset.csv', sep = ';', index = False, encoding = 'cp1251')